In [47]:
import xarray as xr
from ipywidgets import interact
from Magics import macro as magics
import requests

In [38]:
anomalies = xr.open_dataset('anomalies.1980-2018.nc')

In [68]:

def cities(name):
    r = requests.post("https://apps.ecmwf.int/backend/city-lookup/", data= {"name" : name })
    return r
cities("lilia")


<Response [500]>

In [63]:
def get_stripes(minval, maxval):

    
    x = max(abs(minval), abs(maxval))
    
    val1 = -x
    val2 = x
    
    stripes_red = ["#67000D", "#A50F15", "#CB181D", "#EF3B2C","#FB694A", "#FC9272", 
               "#FDBBA1", "#FEE0D2"]
    stripes_blue = ["#DEEBF7", "#C6DBEF", "#9DCAE1", "#6CAED6", "#4292C6",
                  "#2271B5", "#06519C", "#09306B"]
    
    width = val2
    step = width / (len(stripes_red) +1 )
    
    levels = []
    for i in range(len(stripes_blue)):
        levels.append(val1 + i *step)

    width = val2
    step = width / (len(stripes_red) +1)


    for i in range(len(stripes_blue)):
        levels.append(i *step)
    levels.append(maxval)
    
     
    stripes = stripes_red + stripes_blue
    stripes = stripes[::-1]
    return (levels, stripes)


In [40]:
def graph(y, val, levels, stripes):
    
    colour = stripes[0]
    for i, col in zip(levels, stripes):
        if val < i:
            break
        colour = col
    
    data = magics.minput(input_x_values = [float(y)],
                    input_y_values = [20.])
    graph = magics.mgraph(graph_bar_colour = colour,
                graph_bar_width = 1.,
                graph_bar_line_colour = colour,
                graph_type = "bar")
    return (data, graph) 

In [41]:
def pixel(lat, lon):
    x = ( lon /360 ) *4000
    y =   2000 + (( (-90-lat)/180) * 2000)
    return (x, y)

In [70]:
#Setting the cartesian view
def warming_stripe(lat, lon, text):
    tlon = lon
    if lon < 0 :
        lon = lon + 360
    x, y = pixel(lat, lon)
    value = anomalies.sel(latitude=lat, longitude=lon)
    projection = magics.mmap(subpage_map_projection = 'cartesian',
                subpage_x_axis_type = 'regular',
                subpage_y_length = 8.,
                subpage_y_position = 12.,
                subpage_x_length = 17.,
                subpage_y_axis_type = 'regular',
                subpage_y_min = 0.,
                subpage_y_max = 20.,
                subpage_x_min = 1979.5,
                subpage_x_max = 2017.5)

    #Vertical axis
    vertical = magics.maxis(axis_orientation = "vertical",
                     axis_grid =  "on",
                     axis_tick_label =  "off",
                     axis_tick =  "off",
                     axis_grid_colour = "grey",
                     axis_grid_thickness = 1,
                     axis_grid_line_style = "dot")

    #Horizontal axis
    horizontal = magics.maxis(axis_orientation = "horizontal",
                     axis_tick_label =  "off",
                     axis_tick =  "off",
                     axis_grid =  "on",
                     axis_grid_colour = "grey",
                     axis_grid_thickness = 1,
                     axis_grid_line_style = "dot")


    toplot = [projection, vertical, horizontal]
    values = value.t2m.values
    
    maxvalue = max(value.t2m.values)
    minvalue = min(value.t2m.values)
    info = ("lat/lon=%.2f/%.2f pixels=[%d/%d] min=%.2f max=%.2f" % (lat, tlon, x, y, minvalue, maxvalue))
    text = "%s [%s]" % (text, info)
    title = magics.mtext(text_lines = [text])
    levels, stripes = get_stripes(minvalue, maxvalue)
    for i, val in zip(range(1980, 2019), value.t2m.values):
        objects = graph(i, val, levels, stripes)
        for o in objects:
            toplot.append(o)
    toplot.append(title)
    out.clear_output()
    with out:
        display(magics.plot(toplot))

        
out = widgets.Output(layout={'border': '1px solid black', 'height':'800px', 'width':'1200px'})
lat = widgets.FloatSlider(description='Latitude', min = -90, max = 90, step = 0.25)
lon = widgets.FloatSlider(description='Longitude', min = -180, max = 180, step = 0.25)
text = widgets.Text(description='City', value="")
apply = widgets.Button(description='Get the Stripes')


def get_the_stripes(change):
    print (change)
    warming_stripe(lat.value, lon.value, text.value)

apply.on_click(get_the_stripes)


display(lat,lon, text, apply)
display(out)

FloatSlider(value=0.0, description='Latitude', max=90.0, min=-90.0, step=0.25)

FloatSlider(value=0.0, description='Longitude', max=180.0, min=-180.0, step=0.25)

Text(value='', description='City')

Button(description='Get the Stripes', style=ButtonStyle())

Output(layout=Layout(border='1px solid black', height='800px', width='1200px'))